In [1]:
from Pyro5.api import expose, Daemon, locate_ns
from os import listdir
import threading
import pandas as pd
from os import path
import netifaces
import glob
from DirectoryGenerator import DirectoryGenerator

In [ ]:
class ClientManager(Daemon):
    def __init__(this, host, port):
        super().__init__(host, port)
        this.numberOfClients = 0
        
    def validateHandshake(this, conn, data):
        this.numberOfClients = this.numberOfClients + 1
        #print(this.numberOfClients)
    
    def clientDisconnect(this, conn):
        this.numberOfClients = this.numberOfClients - 1
        #print(this.numberOfClients)
        if(this.numberOfClients == 0):
            this.close()

In [ ]:
class csvWriter:
    def __init__(this, fileName):
        this.fileName = fileName
        this.headerNotWritten = True
    
    def appendData(this, data):
        def checkHeader(lock):
            lock.acquire()
            res = this.headerNotWritten
            this.headerNotWritten = False
            lock.release()
            return res
        
        data.to_csv(this.fileName, mode = 'a', header = checkHeader(threading.Lock()), index = False)

In [2]:
@expose
class DataWarehouse(object):
    def __init__(self, dirpath, outfile, dataCleanFunc):
        dirGen = DirectoryGenerator()
        self.dirpath = dirpath
        self.contents = glob.glob(dirpath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.jsonl', recursive=True)
        self.outfileWriter = csvWriter(outfile + '.csv')
        self.dataCleanFunc = dataCleanFunc

    def removeItem(self):
        def removeItemProper(lock):
            lock.acquire()
            item = None
            try:
                item = self.contents.pop()
            except(IndexError):
                item = None
            lock.release()
            return item
        
        return removeItemProper(threading.Lock())
    
    def cleanseData(self, item):
        result = self.dataCleanFunc(item)
        self.outfileWriter.appendData(result)

In [3]:
def openDataWarehouse(dirpath, outfile, dataCleanFunc):
    def getWiFiIPAdd():
        iface = netifaces.gateways()['default'][netifaces.AF_INET][1]
        ipadd = netifaces.ifaddresses(iface)[netifaces.AF_INET][0]['addr']
        return ipadd
    
    datawarehouse = DataWarehouse(dirpath, outfile, dataCleanFunc)
    HOST_IP = getWiFiIPAdd()
    HOST_PORT = 9092
    with ClientManager(host=HOST_IP, port=HOST_PORT) as daemon:
        datawarehouse_uri = daemon.register(datawarehouse)
        with locate_ns() as ns:
            ns.register("example.datawarehouse", datawarehouse_uri)
        daemon.requestLoop()